In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import glob
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import geoviews.tile_sources as gts
from cartopy import crs
import re
from datetime import datetime

In [ ]:
gv.extension('bokeh')
%output size=300

In [ ]:
rsync -v --progress  --exclude provenance -a /mnt/oneclient/eWaterCycle/forecast-results/forecasts ~/

In [ ]:
files = glob.glob('/mnt/home/student1/forecasts/*/member*-discharge_dailyTot_output.nc')

In [ ]:
ensemble_members = [xr.open_dataset(fn) for fn in files]

In [ ]:
forecast_files = {}
for fn in files:
    (y, m, d, ensembl_id) = re.search(r'forecasts/(\d{4})(\d{2})(\d{2})/member(.+)-', fn).groups()
    ensembl_id = int(ensembl_id)
    forecast = datetime(int(y), int(m) ,int(d))
    if forecast in forecast_files:
        forecast_files[forecast][ensembl_id] = xr.open_dataset(fn)
    else:
        forecast_files[forecast] = {ensembl_id: xr.open_dataset(fn)}

In [ ]:
forecasts = []
forecasts_startdates = []
for dt, ensemble_members in forecast_files.items():
    ensemble = xr.concat(ensemble_members.values(), pd.Index(ensemble_members.keys(), name='ensemble_id'))
    forecasts.append(ensemble)
    forecasts_startdates.append(dt)

In [ ]:
ds = xr.concat(forecasts, pd.Index(forecasts_startdates, name='forecast'))

In [59]:
ds

<xarray.Dataset>
Dimensions:      (ensemble_id: 20, forecast: 8, lat: 360, lon: 720, time: 16)
Coordinates:
  * time         (time) datetime64[ns] 2018-11-01 2018-11-02 2018-11-03 ...
  * lat          (lat) float32 89.75 89.25 88.75 88.25 87.75 87.25 86.75 ...
  * lon          (lon) float32 -179.75 -179.25 -178.75 -178.25 -177.75 ...
  * ensemble_id  (ensemble_id) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
  * forecast     (forecast) datetime64[ns] 2018-11-05 2018-11-07 2018-11-08 ...
Data variables:
    discharge    (forecast, ensemble_id, time, lat, lon) float32 nan nan nan ...
Attributes:
    description:  test version (by Niels Drost)
    institution:  Department of Physical Geography, Utrecht University
    title:        PCR-GLOBWB output